# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)


## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **restaurant** in **Toronto**, Canada.

Here we will try finding if someone wants to open a new restaurant in the city which location is best suited for it keeping in mind the competitors and which income group of people will be attracted most to it based on the **population of the neighbourhood**.

Since there are lots of restaurants in Toronto we will try to detect **locations that are not already crowded with restaurants**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* All existing restaurants in the neighborhood (any type of restaurant)
* Age group of people with their income
* Distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)


In [2]:
# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 

Nebr = pd.DataFrame(columns=column_names)

## 1. Download and Explore Dataset

In [3]:
from urllib.request import urlopen
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

page = urlopen(wiki)

from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":906439794,"wgRevisionId":906439794,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June",

In [4]:
Toronto=soup.find('table', class_='wikitable sortable')
Toronto

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [5]:
#Generate lists
Pos=[]
Bor=[]
Neig=[]

for row in Toronto.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: 
        Pos.append(cells[0].find(text=True))
        Bor.append(cells[1].find(text=True))
        Neig.append(cells[2].find(text=True))

        
#Add Data to our DataFrame
Nebr['Postal_Code']=Pos
Nebr['Borough']=Bor
Nebr['Neighborhood']=Neig

Nebr

,Postal_Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Data Cleaning

If Borough is Not Assigned drop row.

Reset Index

In [6]:
Nebr = Nebr.drop(Nebr[Nebr['Borough'].str.contains("Not assigned")==True].index, axis=0, inplace=False)

Nebr.index = pd.RangeIndex(len(Nebr.index))
Nebr

,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [7]:
Nebr.shape

(211, 3)

In [8]:
column_n = ['NEBRVAL'] 
NEBR_NA = pd.DataFrame(columns=column_n)
NA=['Not assigned'] 
NEBR_NA['NEBRVAL'] = NA

Nebr1=Nebr

for row_index,row in Nebr.iterrows():
    if((Nebr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned') or (Nebr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned\n')):
       
        Nebr1.loc[row_index,['Neighborhood']] = Nebr1.loc[row_index,['Borough']].values.astype('str') 
        
#Reset Index
#nbr.index = pd.RangeIndex(len(nbr.index))

Nebr1

,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [9]:
Nebr2=Nebr1.groupby('Postal_Code').agg({'Borough':'first',
                               'Neighborhood': ', '.join}).reset_index()

column_names = ['Postal_Code','Borough', 'Neighborhood'] 
Nebr3 = pd.DataFrame(columns=column_names)

Nebr3 = Nebr2.drop(Nebr2[Nebr2['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
Nebr3.index = pd.RangeIndex(len(Nebr3.index))
Nebr3

,Postal_Code,Borough,Neighborhood
0,M4E,East Toronto,The Beaches
1,M4K,East Toronto,"The Danforth West\n, Riverdale"
2,M4L,East Toronto,"The Beaches West\n, India Bazaar"
3,M4M,East Toronto,Studio District\n
4,M4N,Central Toronto,Lawrence Park
5,M4P,Central Toronto,Davisville North\n
6,M4R,Central Toronto,North Toronto West\n
7,M4S,Central Toronto,Davisville\n
8,M4T,Central Toronto,"Moore Park, Summerhill East\n"
9,M4V,Central Toronto,"Deer Park, Forest Hill SE\n, Rathnelly, South ..."


In [10]:
column = ['Postal_Code','Borough', 'Neighborhood'] 
Nebr_ungrp = pd.DataFrame(columns=column_names)

Nebr_ungrp = Nebr1.drop(Nebr1[Nebr1['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)


Nebr_ungrp.index = pd.RangeIndex(len(Nebr_ungrp.index))
Nebr_ungrp

,Postal_Code,Borough,Neighborhood
0,M5A,Downtown Toronto,Harbourfront
1,M5A,Downtown Toronto,Regent Park
2,M5B,Downtown Toronto,Ryerson
3,M5B,Downtown Toronto,Garden District
4,M5C,Downtown Toronto,St. James Town
5,M4E,East Toronto,The Beaches
6,M5E,Downtown Toronto,Berczy Park
7,M5G,Downtown Toronto,Central Bay Street
8,M6G,Downtown Toronto,Christie
9,M5H,Downtown Toronto,Adelaide


In [11]:
#!conda install -c conda-forge geopy --yes 
import time
from geopy.geocoders import Nominatim

In [12]:
from geopy.util import get_version
get_version()

'1.18.1'

In [13]:
geolocator = Nominatim(scheme='http', user_agent="ES1234")

for row_index, item in Nebr_ungrp.iterrows():
    
    list1 = Nebr_ungrp.loc[[row_index],['Neighborhood']].values.astype('str')
    loc = ' , Toronto, Ontario, Canada'
    list1.astype('str')
    list1 = np.append(list1, loc)
    latitude = None
    longitude = None
    location = None
    
    location = geolocator.geocode(list1 , limit = 15)
    time.sleep(5)
    if(location is not None):
        Nebr_ungrp.loc[Nebr_ungrp.index[row_index], 'Latitude'] = location.latitude
        Nebr_ungrp.loc[Nebr_ungrp.index[row_index], 'Longitude'] = location.longitude
          

In [14]:
print(Nebr_ungrp)

   Postal_Code           Borough                                       Neighborhood   Latitude  Longitude
0          M5A  Downtown Toronto                                       Harbourfront  43.640080 -79.380150
1          M5A  Downtown Toronto                                        Regent Park  43.660706 -79.360457
2          M5B  Downtown Toronto                                           Ryerson
  43.657474 -79.377905
3          M5B  Downtown Toronto                                   Garden District
  43.656502 -79.377128
4          M5C  Downtown Toronto                                     St. James Town  43.669403 -79.372704
5          M4E      East Toronto                                        The Beaches  43.671024 -79.296712
6          M5E  Downtown Toronto                                        Berczy Park  43.647989 -79.375393
7          M5G  Downtown Toronto                                Central Bay Street
        NaN        NaN
8          M6G  Downtown Toronto              

In [16]:
import json 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Importing to use the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes  #Uncomment if not installed
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-3.1.0               |           py36_0         724 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [17]:
print('We have {} boroughs and {} neighborhoods.'.format(
        len(Nebr_ungrp['Borough'].unique()),
        Nebr_ungrp.shape[0]
    )
)

Nebr_ungrp.dropna(inplace =True)
Nebr_ungrp.index = pd.RangeIndex(len(Nebr_ungrp.index))

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ES1234")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

We have 4 boroughs and 74 neighborhoods.
The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [19]:
Nebr_ungrp

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
1,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
2,M5B,Downtown Toronto,Ryerson,43.657474,-79.377905
3,M5B,Downtown Toronto,Garden District,43.656502,-79.377128
4,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704
5,M4E,East Toronto,The Beaches,43.671024,-79.296712
6,M5E,Downtown Toronto,Berczy Park,43.647989,-79.375393
7,M6G,Downtown Toronto,Christie,43.664111,-79.418405
8,M5H,Downtown Toronto,Adelaide,43.651923,-79.370879
9,M5H,Downtown Toronto,King,43.648949,-79.377754


In [20]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Nebr_ungrp['Latitude'], Nebr_ungrp['Longitude'], Nebr_ungrp['Borough'], Nebr_ungrp['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Lets use FOURSQUARE API to explore the neighbourhood

In [21]:
CLIENT_ID = 'QEF4SFROUUVOQKNPAFNNRDUW4ACAWSYYG312LM3BKEDDKPIZ' # your Foursquare ID
CLIENT_SECRET = 'CCBFW1WDVTUYDLPFX3SR0QBB5R2UKRYOZ3F1JDJ3PLQMPDRM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Successfully Logged-In')

Successfully Logged-In


In [22]:
Nebr_ungrp.loc[0]
neighborhood_latitude = np.float(Nebr_ungrp.loc[0,['Latitude']].values)
neighborhood_longitude =  np.float(Nebr_ungrp.loc[0,['Longitude']].values)

###  Now, let's get the top 100 venues that are in Harbour Square Park within a radius of 500 meters.

###### First, let's create the GET request URL. Name the URL url.

In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d34a1ff6c0aa50023fc3639'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Harbourfront',
  'headerFullLocation': 'Harbourfront, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 123,
  'suggestedBounds': {'ne': {'lat': 43.6445801045, 'lng': -79.37394296546611},
   'sw': {'lat': 43.635580095499996, 'lng': -79.38635603453389}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e49413e81dc766f3e3d6312',
       'name': 'Harbour Square Park',
       'location': {'address': '25 Queens Quay West',
        'lat': 43.63925269130776,
        'lng': -79.37839547902978,
        'labeledLatLngs': [{'label': 'display',
          'lat

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Harbour Square Park,Park,43.639253,-79.378395
1,Lake Ontario,Lake,43.638945,-79.379665
2,Harbourfront,Neighborhood,43.639526,-79.380688
3,BeaverTails,Bakery,43.639899,-79.380197
4,Miku,Japanese Restaurant,43.641374,-79.377531


And how many venues were returned by Foursquare?

In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
toronto_venues = getNearbyVenues(names=Nebr_ungrp['Neighborhood'],
                                   latitudes=Nebr_ungrp['Latitude'],
                                   longitudes=Nebr_ungrp['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson

Garden District

St. James Town
The Beaches
Berczy Park
Christie

Adelaide

King

Richmond

Dovercourt Village
Dufferin

Harbourfront East

Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West

Riverdale
Design Exchange
Toronto Dominion Centre
Brockton

Exhibition Place
Parkdale Village
The Beaches West

India Bazaar
Commerce Court
Lawrence Park
Roselawn

Davisville North

Forest Hill North
High Park
The Annex
Yorkville
Parkdale
Roncesvalles
Davisville

Harbord

University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East

Chinatown
Grange Park
Kensington Market
Deer Park
Rathnelly
South Hill
Summerhill West

CN Tower
Bathurst Quay

Harbourfront West

King and Spadina
South Niagara
Rosedale
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


In [30]:
print(toronto_venues.shape)
toronto_venues.head()

(3418, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.64008,-79.38015,Harbour Square Park,43.639253,-79.378395,Park
1,Harbourfront,43.64008,-79.38015,Lake Ontario,43.638945,-79.379665,Lake
2,Harbourfront,43.64008,-79.38015,Harbourfront,43.639526,-79.380688,Neighborhood
3,Harbourfront,43.64008,-79.38015,BeaverTails,43.639899,-79.380197,Bakery
4,Harbourfront,43.64008,-79.38015,Miku,43.641374,-79.377531,Japanese Restaurant


In [31]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,96,96,96,96,96,96
Bathurst Quay,24,24,24,24,24,24
Berczy Park,100,100,100,100,100,100
Brockton,19,19,19,19,19,19
CN Tower,87,87,87,87,87,87
Cabbagetown,50,50,50,50,50,50
Chinatown,100,100,100,100,100,100
Christie,60,60,60,60,60,60
Church and Wellesley,91,91,91,91,91,91


In [32]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 286 uniques categories.


## 3. Analyze Each Neighborhood

In [33]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,

In [34]:
toronto_onehot.shape

(3418, 286)

In [35]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,Accessories Store,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,

In [36]:
toronto_grouped.shape

(61, 286)

Lets Check top Venues

In [37]:
Top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(Top_venues))
    print('\n')

----Adelaide
----
                venue  freq
0         Coffee Shop  0.11
1                Café  0.04
2  Italian Restaurant  0.04
3          Restaurant  0.04
4       Grocery Store  0.03


----Bathurst Quay
----
              venue  freq
0       Coffee Shop  0.21
1              Café  0.12
2              Park  0.08
3  Ramen Restaurant  0.04
4            Tunnel  0.04


----Berczy Park----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2   Restaurant  0.05
3        Hotel  0.05
4       Bakery  0.04


----Brockton
----
                   venue  freq
0                    Bar  0.16
1                   Park  0.11
2  Vietnamese Restaurant  0.11
3      Korean Restaurant  0.05
4  Portuguese Restaurant  0.05


----CN Tower----
                venue  freq
0         Coffee Shop  0.09
1               Hotel  0.09
2         Pizza Place  0.06
3  Italian Restaurant  0.05
4            Aquarium  0.05


----Cabbagetown----
         venue  freq
0   Restaurant  0.08
1         Café  0.06
2  Coff

In [38]:
def return_most_common_venues(row, Top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:Top_venues]

In [60]:
Top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(Top_venues):
    try:
        columns.append('{}{} Popular Venues'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Popular Venues'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], Top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
0,Adelaide,Coffee Shop,Café,Restaurant,Italian Restaurant,Grocery Store,Pizza Place,Cocktail Bar,Diner,Cosmetics Shop,Bakery
1,Bathurst Quay,Coffee Shop,Café,Park,Garden,Pub,Grocery Store,Ramen Restaurant,Japanese Restaurant,Sculpture Garden,Tunnel
2,Berczy Park,Coffee Shop,Café,Restaurant,Hotel,Bakery,Beer Bar,Japanese Restaurant,Gym,Gastropub,Seafood Restaurant
3,Brockton,Bar,Park,Vietnamese Restaurant,South American Restaurant,Gastropub,Bakery,Korean Restaurant,Dive Bar,Grocery Store,Café
4,CN Tower,Hotel,Coffee Shop,Pizza Place,Italian Restaurant,Aquarium,Gym,Scenic Lookout,Baseball Stadium,Bar,Concert Hall
5,Cabbagetown,Restaurant,Café,Coffee Shop,Pub,Bakery,Gastropub,Diner,Japanese Restaurant,Italian Restaurant,Pizza Place
6,Chinatown,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Vietnamese Restaurant,Dumpling Restaurant,Mexican Restaurant,Coffee Shop,Bakery,Dessert Shop
7,Christie,Korean Restaurant,Coffee Shop,Indian Restaurant,Cocktail Bar,Café,Japanese Restaurant,Karaoke Bar,Dessert Shop,Sandwich Place,Grocery Store
8,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Café,Restaurant,Burger Joint,Gym,Men's Store,Mediterranean Restaurant
9,Commerce Court,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Steakhouse,Japanese Restaurant,Italian Restaurant,Bakery,Gastropub


## 4. Cluster Neighborhoodsusing K-Mean

In [68]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)
#print(toronto_grouped_clustering)
#print(toronto_grouped)
# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
labels = kmeans.labels_[0:63] 
print(labels)

[0 0 0 1 0 3 1 1 0 0 3 3 3 0 0 1 0 0 4 0 0 0 0 0 0 1 1 1 0 0 3 1 4 1 1 3 3
 0 1 3 4 0 3 0 3 0 0 3 3 2 3 3 3 1 0 3 0 3 0 1 0]


In [69]:
toronto_merged = Nebr_ungrp
print(toronto_merged.shape)
labels = np.append(labels,labels[0])
print(labels.shape)
# add clustering labels
toronto_merged['Cluster Labels'] = labels.tolist()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

(62, 6)
(62,)


,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
0,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150,0,Coffee Shop,Café,Restaurant,Hotel,Pizza Place,Italian Restaurant,Steakhouse,History Museum,Sporting Goods Shop,Park
1,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,0,Coffee Shop,Thai Restaurant,Pharmacy,Grocery Store,Food Truck,Moving Target,Beer Store,Electronics Store,Sushi Restaurant,Restaurant
2,M5B,Downtown Toronto,Ryerson,43.657474,-79.377905,0,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Café,Middle Eastern Restaurant,Diner,Tea Room,Bookstore,Plaza
3,M5B,Downtown Toronto,Garden District,43.656502,-79.377128,1,Clothing Store,Coffee Shop,Cosmetics Shop,Fast Food Restaurant,Restaurant,Hotel,Lingerie Store,Middle Eastern Restaurant,Tea Room,Theater
4,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704,0,Coffee Shop,Pizza Place,Grocery Store,Café,Beer Store,Bakery,Bank,Restaurant,Library,Breakfast Spot


In [70]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
0,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Pizza Place,Italian Restaurant,Steakhouse,History Museum,Sporting Goods Shop,Park
1,Downtown Toronto,0,Coffee Shop,Thai Restaurant,Pharmacy,Grocery Store,Food Truck,Moving Target,Beer Store,Electronics Store,Sushi Restaurant,Restaurant
2,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Café,Middle Eastern Restaurant,Diner,Tea Room,Bookstore,Plaza
4,Downtown Toronto,0,Coffee Shop,Pizza Place,Grocery Store,Café,Beer Store,Bakery,Bank,Restaurant,Library,Breakfast Spot
8,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Grocery Store,Pizza Place,Cocktail Bar,Diner,Cosmetics Shop,Bakery
9,Downtown Toronto,0,Coffee Shop,Restaurant,Hotel,Café,Italian Restaurant,Gastropub,American Restaurant,Gym,Japanese Restaurant,Deli / Bodega
13,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Pizza Place,Italian Restaurant,Steakhouse,History Museum,Sporting Goods Shop,Park
14,Downtown Toronto,0,Playground,Farm,Scenic Lookout,American Restaurant,Beer Garden,Beach,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
16,West Toronto,0,Bar,Coffee Shop,Café,Restaurant,Cocktail Bar,Bakery,Thai Restaurant,Portuguese Restaurant,French Restaurant,Korean Restaurant
17,West Toronto,0,Clothing Store,Coffee Shop,Cosmetics Shop,Restaurant,Plaza,Tea Room,Fast Food Restaurant,Lingerie Store,Pizza Place,Theater


#### Cluster 2

In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
3,Downtown Toronto,1,Clothing Store,Coffee Shop,Cosmetics Shop,Fast Food Restaurant,Restaurant,Hotel,Lingerie Store,Middle Eastern Restaurant,Tea Room,Theater
6,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Beer Bar,Japanese Restaurant,Gym,Gastropub,Seafood Restaurant
7,Downtown Toronto,1,Korean Restaurant,Coffee Shop,Indian Restaurant,Cocktail Bar,Café,Japanese Restaurant,Karaoke Bar,Dessert Shop,Sandwich Place,Grocery Store
15,Downtown Toronto,1,Coffee Shop,Café,Deli / Bodega,Restaurant,Hotel,Bakery,Italian Restaurant,Breakfast Spot,Steakhouse,Gym
25,East Toronto,1,Beach,Bar,Breakfast Spot,Japanese Restaurant,Tea Room,Coffee Shop,Pub,Thai Restaurant,Park,Salon / Barbershop
26,East Toronto,1,Indian Restaurant,Café,Halal Restaurant,Grocery Store,Snack Place,Pakistani Restaurant,Sandwich Place,Restaurant,Egyptian Restaurant,Discount Store
27,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Steakhouse,Japanese Restaurant,Italian Restaurant,Bakery,Gastropub
31,Central Toronto,1,Playground,Bank,Park,Mediterranean Restaurant,Yoga Studio,Farm,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
33,Central Toronto,1,Pizza Place,Coffee Shop,Park,Thai Restaurant,Bistro,Grocery Store,Ice Cream Shop,Indian Restaurant,Falafel Restaurant,Fried Chicken Joint
34,Central Toronto,1,Café,Boutique,Coffee Shop,Restaurant,Italian Restaurant,French Restaurant,Sushi Restaurant,Spa,Hotel,Women's Store


#### Cluster 3

In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
49,Central Toronto,2,Coffee Shop,Sandwich Place,History Museum,Pizza Place,Café,Jewish Restaurant,Cosmetics Shop,Park,Pub,Cheese Shop


#### Cluster 4

In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
5,East Toronto,3,Beach,Bar,Breakfast Spot,Japanese Restaurant,Tea Room,Coffee Shop,Pub,Thai Restaurant,Park,Salon / Barbershop
10,Downtown Toronto,3,Coffee Shop,Hotel,Restaurant,Diner,Gastropub,Cosmetics Shop,Cocktail Bar,Clothing Store,Italian Restaurant,Pizza Place
11,West Toronto,3,Café,Restaurant,Pizza Place,Bar,Park,Bus Line,Coffee Shop,Falafel Restaurant,Electronics Store,Ethiopian Restaurant
12,West Toronto,3,Bar,Bakery,Coffee Shop,Café,Cocktail Bar,Sandwich Place,Clothing Store,Beer Bar,Beer Store,Mexican Restaurant
30,Central Toronto,3,Sandwich Place,Dessert Shop,Coffee Shop,Sushi Restaurant,Pizza Place,Thai Restaurant,Café,Italian Restaurant,Fried Chicken Joint,Seafood Restaurant
35,West Toronto,3,Tibetan Restaurant,Bar,Café,Pharmacy,Diner,Restaurant,Indian Restaurant,Liquor Store,North Indian Restaurant,Donut Shop
36,West Toronto,3,Bookstore,Gourmet Shop,Breakfast Spot,Restaurant,Coffee Shop,Gift Shop,Café,Gastropub,Bank,Movie Theater
39,Downtown Toronto,3,Café,Bookstore,Park,Japanese Restaurant,Yoga Studio,College Gym,Museum,French Restaurant,Noodle House,Chinese Restaurant
42,Central Toronto,3,Playground,Gym,Tennis Court,Park,Yoga Studio,Event Space,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant
44,Downtown Toronto,3,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Vietnamese Restaurant,Dumpling Restaurant,Mexican Restaurant,Coffee Shop,Bakery,Dessert Shop


#### Cluster 5

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
18,East Toronto,4,Grocery Store,Pharmacy,Coffee Shop,Bus Line,Doctor's Office,Optical Shop,Sandwich Place,Caribbean Restaurant,Café,Skating Rink
32,West Toronto,4,Baseball Field,Dog Run,Zoo,Garden,Other Great Outdoors,Park,Scenic Lookout,Skating Rink,Café,Pool
40,West Toronto,4,Pizza Place,Rental Car Location,Coffee Shop,Pharmacy,Grocery Store,Café,Fried Chicken Joint,Sandwich Place,Bar,Thai Restaurant
